In [1]:
import pandas as pd
import numpy as np
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator
h=6
window=24
sources=['instagram-content','twitter-tweets','tiktok-content','facebook-content','youtube-content',
 'tiktok-content-oauth','instagram-content-oauth','tiktok-content-public']
df= pd.read_csv('~/rug/thesis/thesis/data/influencer_sample010525_transformed.csv').query('source in @sources').reset_index(drop=True)
df['published_date'] = pd.to_datetime(df['publishedDate'])
df


/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,post_uid,channel_uid,publishedDate,metrics,potentialReach,source,comments,video_views,engagements,video_plays,shares,engagements_rate,audience,tier,predicted,published_date
0,000075b79f9c314c9dbe766d07016ef0,c149b6fafe123318b768e2e1c716abeb,2022-11-14T16:11:50.000Z,"[{""name"":""facebook_comments"",""value"":""0""},{""na...",21706.0,facebook-content,0,0.0,9,0.0,NaN,0.000415,21706.0,"10,000–49,999",0.026351,2022-11-14 16:11:50+00:00
1,0001eb4819c534a7a5315eac3ef96e2d,96b6431e916f3f629483d6fa06acf335,2024-11-23T03:01:44.000Z,"[{""name"":""tiktok_video_shares"",""value"":""22""},{...",140500.0,tiktok-content,15,0.0,1073,157200.0,22.0,0.007637,140500.0,"100,000–499,999",0.024644,2024-11-23 03:01:44+00:00
2,0001f8ab1ee338b98597bbce534a4b1a,570ef814e5983fc8a152009d2be91bef,2019-08-16T16:06:52.000Z,"[{""name"":""facebook_reactions"",""value"":""120""},{...",69481.0,facebook-content,10,0.0,130,0.0,1.0,0.001871,69481.0,"50,000–99,999",0.025664,2019-08-16 16:06:52+00:00
3,000279c0e7a23a8abefbbbb49c740eb6,b95a899244573cf38802aa520ef3a82b,2021-08-18T15:00:42.000Z,"[{""name"":""instagram_comments"",""value"":""6""},{""n...",82089.0,instagram-content,6,0.0,4838,NaN,NaN,0.058936,82089.0,"50,000–99,999",0.025483,2021-08-18 15:00:42+00:00
4,000295877e633a7d946cb1ddde240f39,da9acec7669f3db79050175948471cde,2020-12-11T17:36:08.000Z,"[{""name"":""twitter_likes"",""value"":""7""},{""name"":...",28829.0,twitter-tweets,0,0.0,7,0.0,0.0,0.000243,28829.0,"10,000–49,999",0.026249,2020-12-11 17:36:08+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218720,fffacd0414c23b0eb9f66827f31d8e4f,cd186be335983c119a4e15d490ff4807,2024-01-21T12:57:43.000Z,"[{""name"":""tiktok_video_shares"",""value"":""7""},{""...",1052.0,tiktok-content,0,0.0,3,248.0,7.0,0.002852,1052.0,"1,000–9,999",0.026648,2024-01-21 12:57:43+00:00
218721,fffd19c0afa4399c81655363cee09985,e04c6702e00a321a8f98624ffe7fd23e,2024-08-26T00:04:44.000Z,"[{""name"":""tiktok_video_shares"",""value"":""2""},{""...",1900000.0,tiktok-content,9,0.0,224,15800.0,2.0,0.000118,1900000.0,"500,000+",-0.000647,2024-08-26 00:04:44+00:00
218722,fffe71ee0ced38a49a2cead83aefc0c5,26bf145cbe9c3a10a53c619f05aa750f,2023-06-19T20:42:43.000Z,"[{""name"":""instagram_comments"",""value"":""1""},{""n...",251921.0,instagram-content,1,NaN,44,5628.0,NaN,0.000175,251921.0,"100,000–499,999",0.023042,2023-06-19 20:42:43+00:00
218723,fffee021bda236f6a839c60fe862307a,8ed4ca5c384d3857a7134e0b93acf87b,2022-07-27T14:43:26.000Z,"[{""name"":""youtube_likes"",""value"":""0""},{""name"":...",18.0,youtube-content,0,1.0,0,0.0,0.0,0.000000,18.0,"less than 1,000",0.026663,2022-07-27 14:43:26+00:00


In [2]:
df['source'].value_counts()

source
instagram-content    119688
twitter-tweets        50484
tiktok-content        36721
facebook-content       9096
youtube-content        2736
Name: count, dtype: int64

In [3]:
df.groupby('channel_uid').count()['publishedDate'].sort_values(ascending=False).head(10)

channel_uid
26bf145cbe9c3a10a53c619f05aa750f    32490
1058025cd111366f9d4d4af89ce1d8d0    24051
954328679b923a4fa7af327a2cb5b84d    13640
da9acec7669f3db79050175948471cde     9714
e04c6702e00a321a8f98624ffe7fd23e     6232
24a13f11774b37b2bb94e9ab25327bf7     4884
e6981573da773e0eb0493a0cf142e27c     4274
428271c836453fc2a6fd57748371c14d     3608
1fc2d888a1d63daea81ab67327629a64     3478
570ef814e5983fc8a152009d2be91bef     3352
Name: publishedDate, dtype: int64

In [ ]:
ids_test=df.query("""source in ['instagram-content','youtube-content','tiktok-content','facebook-content']""")\
    ['channel_uid'].drop_duplicates().sample(1, random_state=1).values
#df_temp=df.query('channel_uid in @ids_test')
df_temp=df.copy()
df_temp['source'].unique()

array(['instagram-content'], dtype=object)

In [5]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid')

In [6]:

multi_eval.run(h=6, initial_window=24, step_length=6,models=['MA(5)'],metric_cols=['engagements_rate'])

summary_df = multi_eval.get_summary()
summary_complete_df = multi_eval.get_summary_complete()

Evaluating models:   0%|          | 0/1 [00:00<?, ?it/s]2025-05-12 10:07:17,941 Evaluating channel: 51b52b775df43f1ea0abf5db8789446d
2025-05-12 10:07:17,951 Evaluating metric: engagements_rate
2025-05-12 10:07:17,953 Evaluating MA(5) for 51b52b775df43f1ea0abf5db8789446d/engagements_rate
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thes

In [7]:
import datetime as dt
summary_df.to_csv(f'../output/summary_df_{dt.datetime.now():%y%m%d%H%M}.csv', index=False)
summary_df

,model_name,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,test_MeanSquaredError,channel_uid,metric_name,source
0,MA(5),0.007361,0.341301,0.008799,51b52b775df43f1ea0abf5db8789446d,engagements_rate,instagram-content


In [9]:

summary_complete_df.to_csv(f'../output/summary_complete_df_{dt.datetime.now():%y%m%d%H%M}.csv', index=False)
summary_complete_df

,test_MeanAbsoluteError,test_MeanSquaredError,test_MeanAbsolutePercentageError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred,model_name,channel,metric_name,source
0,0.007196,0.007537,0.289759,2.066471,0.019363,24,23,0 0.021223 1 0.042447 2 0.023720 3...,24 0.021223 25 0.021223 26 0.029963 2...,24 0.009949 25 0.017167 26 0.021795 2...,MA(5),51b52b775df43f1ea0abf5db8789446d,engagements_rate,instagram-content
1,0.006597,0.007253,0.530892,1.376911,0.018057,30,29,0 0.021223 1 0.042447 2 0.023720 3...,30 0.019351 31 0.011860 32 0.011860 3...,30 0.024634 31 0.017826 32 0.020093 3...,MA(5),51b52b775df43f1ea0abf5db8789446d,engagements_rate,instagram-content
2,0.011681,0.015574,0.359904,0.687495,0.022710,36,35,0 0.021223 1 0.042447 2 0.023720 3...,36 0.051810 37 0.044320 38 0.021848 3...,36 0.022940 37 0.021889 38 0.027563 3...,MA(5),51b52b775df43f1ea0abf5db8789446d,engagements_rate,instagram-content
3,0.003970,0.004834,0.184647,1.278696,0.024477,42,41,0 0.021223 1 0.042447 2 0.023720 3...,42 0.016854 43 0.021848 44 0.026217 4...,42 0.014389 43 0.021128 44 0.017258 4...,MA(5),51b52b775df43f1ea0abf5db8789446d,engagements_rate,instagram-content
